In [3]:
import pandas as pd

In [4]:
features = pd.read_csv('ZillowCC01.csv')

In [5]:
features.drop(['Unnamed: 0'], axis=1, inplace=True)
features.head()

,unformattedPrice,addressCity,addressZipcode,beds,baths,area,isHomeRec,relaxed,latLong.latitude,latLong.longitude,...,isPropertyResultCDP,hdpData.homeInfo.group_type,Kluster,radiusAvgBaths,radiusAvgBeds,radiusAvgArea,radiusAreaPercentile,radiusBedsPercentile,radiusBathsPercentile,homesInRadius
0,343000,0,0,4.0,2.0,2146.0,0,0,34.306420,-83.855730,...,-1,-1,93,1.973941,2.752027,1698.269036,0.211387,0.276776,0.371477,1774
1,300000,0,0,4.0,2.0,1581.0,0,0,34.315800,-83.833786,...,-1,-1,93,1.857039,2.613757,1650.515181,0.179012,0.245679,0.322840,1620
2,329900,1,1,4.0,3.0,2369.0,0,0,33.773850,-84.006210,...,-1,-1,43,2.611102,3.333502,1783.053016,0.247053,0.408410,0.568812,6278
3,349000,2,2,4.0,3.0,2988.0,0,0,34.080894,-83.868385,...,-1,-1,133,2.423464,3.238448,1831.290224,0.266185,0.382254,0.526180,3877
4,174000,3,3,2.0,1.0,0.0,1,0,34.423496,-83.100530,...,-1,-1,119,1.764706,2.883002,1431.381995,0.101639,0.126230,0.127869,610


# Linear pearson correlation coefficients

In [8]:
coordf = features[['unformattedPrice', 'beds', 'baths', 'area', 'latLong.latitude', 'latLong.longitude', 'radiusAvgBaths', 'radiusAvgBeds', 'radiusAvgArea', 'radiusAreaPercentile', 'radiusBedsPercentile', 'radiusBathsPercentile', 'homesInRadius', 'Kluster']]
corr = coordf.corr()
corr.style.background_gradient(cmap='coolwarm')

,unformattedPrice,beds,baths,area,latLong.latitude,latLong.longitude,radiusAvgBaths,radiusAvgBeds,radiusAvgArea,radiusAreaPercentile,radiusBedsPercentile,radiusBathsPercentile,homesInRadius,Kluster
unformattedPrice,1.000000,0.405301,0.505830,0.206072,0.125695,-0.046785,0.244590,0.016162,0.016712,0.135042,0.145177,0.308105,0.146357,0.015758
beds,0.405301,1.000000,0.820643,0.394054,-0.072026,-0.006450,0.248267,0.273907,0.132309,0.191497,0.283850,0.217206,0.076921,-0.011621
baths,0.505830,0.820643,1.000000,0.297100,-0.010093,-0.033672,0.295688,0.202132,0.081657,0.192616,0.259918,0.299790,0.140407,0.004146
area,0.206072,0.394054,0.297100,1.000000,-0.023856,0.007961,0.095964,0.126261,0.215008,0.092861,0.108857,0.071021,-0.014127,-0.002715
latLong.latitude,0.125695,-0.072026,-0.010093,-0.023856,1.000000,-0.566867,-0.061013,-0.488669,-0.203928,-0.235305,-0.231528,0.189204,0.290929,0.088290
latLong.longitude,-0.046785,-0.006450,-0.033672,0.007961,-0.566867,1.000000,0.135046,0.325980,0.234120,0.221845,0.212174,-0.093455,-0.308578,-0.120348
radiusAvgBaths,0.244590,0.248267,0.295688,0.095964,-0.061013,0.135046,1.000000,0.640019,0.324056,0.652010,0.743142,0.845757,0.253779,0.015319
radiusAvgBeds,0.016162,0.273907,0.202132,0.126261,-0.488669,0.325980,0.640019,1.000000,0.568175,0.665295,0.798678,0.372896,-0.191908,-0.026343
radiusAvgArea,0.016712,0.132309,0.081657,0.215008,-0.203928,0.234120,0.324056,0.568175,1.000000,0.511260,0.421238,0.166074,-0.380104,-0.002610
radiusAreaPercentile,0.135042,0.191497,0.192616,0.092861,-0.235305,0.221845,0.652010,0.665295,0.511260,1.000000,0.750358,0.599459,-0.158213,0.025196


# Kendall correlation coefficients

In [9]:
corr = coordf.corr('kendall')
corr.style.background_gradient(cmap='coolwarm')

,unformattedPrice,beds,baths,area,latLong.latitude,latLong.longitude,radiusAvgBaths,radiusAvgBeds,radiusAvgArea,radiusAreaPercentile,radiusBedsPercentile,radiusBathsPercentile,homesInRadius,Kluster
unformattedPrice,1.000000,0.399423,0.489025,0.412657,0.129620,-0.045909,0.228860,0.010907,0.044183,0.133753,0.132849,0.280152,0.179851,0.011797
beds,0.399423,1.000000,0.689012,0.610984,-0.079390,0.005120,0.183244,0.212858,0.140815,0.188393,0.227491,0.162510,0.081875,-0.009999
baths,0.489025,0.689012,1.000000,0.621079,-0.003329,-0.026121,0.231295,0.134753,0.096589,0.178793,0.196430,0.244403,0.155504,0.001021
area,0.412657,0.610984,0.621079,1.000000,-0.074121,0.037090,0.170320,0.185074,0.166154,0.186113,0.183464,0.139290,0.023837,-0.000381
latLong.latitude,0.129620,-0.079390,-0.003329,-0.074121,1.000000,-0.305453,-0.098860,-0.398883,-0.264116,-0.192235,-0.202740,0.133180,0.148342,0.028826
latLong.longitude,-0.045909,0.005120,-0.026121,0.037090,-0.305453,1.000000,0.096606,0.227868,0.295746,0.176575,0.169003,-0.036092,-0.195495,-0.076545
radiusAvgBaths,0.228860,0.183244,0.231295,0.170320,-0.098860,0.096606,1.000000,0.426939,0.377769,0.541349,0.558698,0.661051,0.250428,0.007625
radiusAvgBeds,0.010907,0.212858,0.134753,0.185074,-0.398883,0.227868,0.426939,1.000000,0.589478,0.538149,0.618309,0.210222,-0.102261,-0.024524
radiusAvgArea,0.044183,0.140815,0.096589,0.166154,-0.264116,0.295746,0.377769,0.589478,1.000000,0.618747,0.495112,0.208527,-0.225021,-0.008917
radiusAreaPercentile,0.133753,0.188393,0.178793,0.186113,-0.192235,0.176575,0.541349,0.538149,0.618747,1.000000,0.633301,0.448512,0.019655,0.025407


# Feature importance estimated using random forrest

In [37]:
# Use numpy to convert to arrays
import numpy as np# Labels are the values we want to predict
labels = np.array(features['unformattedPrice'])# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('unformattedPrice', axis = 1)# Saving feature names for later use
feature_list = list(features.columns)# Convert to numpy array
features = np.array(features)

In [38]:
from sklearn.model_selection import train_test_split# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

In [39]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (49783, 25)
Training Labels Shape: (49783,)
Testing Features Shape: (16595, 25)
Testing Labels Shape: (16595,)


In [44]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 5000, random_state = 42)# Train the model on training data
rf.fit(train_features, train_labels);

In [45]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)# Calculate the absolute errors
errors = abs(predictions - test_labels)# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 14664.24 degrees.


In [46]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 92.9 %.


In [47]:
# Get numerical feature importances
importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: baths                Importance: 0.37
Variable: area                 Importance: 0.08
Variable: latLong.latitude     Importance: 0.08
Variable: hdpData.homeInfo.livingArea Importance: 0.08
Variable: radiusAvgBeds        Importance: 0.06
Variable: radiusBathsPercentile Importance: 0.04
Variable: latLong.longitude    Importance: 0.03
Variable: hdpData.homeInfo.lotAreaValue Importance: 0.03
Variable: radiusAvgBaths       Importance: 0.03
Variable: radiusAvgArea        Importance: 0.03
Variable: homesInRadius        Importance: 0.03
Variable: addressZipcode       Importance: 0.02
Variable: beds                 Importance: 0.02
Variable: hdpData.homeInfo.homeType Importance: 0.02
Variable: Kluster              Importance: 0.02
Variable: radiusAreaPercentile Importance: 0.02
Variable: radiusBedsPercentile Importance: 0.02
Variable: addressCity          Importance: 0.01
Variable: isHomeRec            Importance: 0.0
Variable: relaxed              Importance: 0.0
Variable: hdpData.ho